In [ ]:
# 아래 import를 위해 선행 설치할 친구들
# !pip install cx_Oracle matplotlib numpy Flask Flask-SocketIO flask-cors eventlet pydub wavio

In [47]:
import os

from flask import flash
# 파일 이름 중복 문제 해결하는 방법
import time
# 파일 업로드시 경로 공격 방지를 막아주는 함수를 제공하는 라이브러리
from werkzeug.utils import secure_filename

# HTTP 요청에서 전송된 이미지 바이트 스트림을 메모리상에서 처리할 때 사용될 수 있습니다.
import io
from io import BytesIO

from flask import Flask, send_file, render_template, request, jsonify, send_from_directory
from flask import session
from flask import redirect
from flask import url_for

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# 소켓 사용시 -> 실시간 정보 이동
from flask_socketio import SocketIO, emit

# 다른 도메인에어 오는 요청을 허용하도록 설정하는 라이브러리
from flask_cors import CORS

# 현시점 날짜, 시간정보 알아내는 라이브러리
from datetime import datetime

# 오라클DB랑 연동
import cx_Oracle

# 임시파일과 디렉토리를 만들고 조작하는데 사용되는 라이브러리
import tempfile

# 컨텍스트 매니저를 만들어주고 효율적이고 간결한 코드 작성 가능
from contextlib import contextmanager

import sys

# flask와 함께 사용되는 웹 서버 라이브러리, 비동기 I/O 지원 -> 빠르고 확장석 있는 웹 어플 개발 가능하답니다.
import eventlet
import eventlet.wsgi

# 문자열 64 인코딩/디코딩용 python내장모델
import base64

# 이미지 detect를 위한 라이브러리
import torch

# flask에서 비동기적으로 명령을 수행하게 해주는 함수를 포함한 라이브러리
import subprocess

# Ajax를 통해 사진 데이터가 넘어올 때 사용할 라이브러리
from PIL import Image

# 없어도 되지만 일단 넣어보는 로직
!git config --global user.email "chj0253@gmail.com"
!git config --global user.name "BusUserYM"

'git'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'git'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [2]:
%pwd

'C:\\Users\\smhrd\\Yolov7_Detect'

In [53]:

txt_path = r'C:/Users/smhrd/Yolov7_Detect/img_results/labels'
file_name = '1'
# 위치만 고정해주면 바로 로직 활용 가능합니다.
with open(f'{txt_path}/{file_name}.txt', 'r') as f:
    data = f.read()
        
data

'asdasdawerascdasdfasdfs'

In [62]:
app = Flask(__name__)
# flask 어플리케이션에서 세션을 사용하려면 반드시 있어야하는 'SECRET_KEY'라고 합니다. 없으면 세션 사용 불가능!
app.config['SECRET_KEY'] = 'today is good day but weather is sooooo bad that make me sad'
CORS(app)

# 이런 식으로 쓸수 있습니다.
@app.route('/')
def index():
    return '실행이 잘 되고 있는가...확인 중!'


# YOLOv7 경로 설정
os.environ["PATH"] += os.pathsep + "C:\\Users\\smhrd\\Yolov7_Detect\\yolov7\\runs\\train\\try3_now_best\\weights\\best.pt"


# flask를 통해 업로드된 파일을 저장한 디렉토리 경로
UPLOAD_FOLDER = 'uploads'
# 아래의 파일 형태만 업로드 가능하다 라는 의미.
ALLOWED_EXTENSIONS = {'jpg','jpeg','JPG','JPEG','png','PNG','GIF','gif'}
# flask에서 아래에 업로드된 값을 'app.config'를 통해 전역 환경 변수로 사용할 수 있게 만드는 로직
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER


# 파일 이름이 허용된 확장자를 가지고 있는지 여부를 판단하는 함수
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS


# 이게 맞네요.
@app.route('/upload-image', methods=['POST'])
def upload():
    image_data = request.files['image']
    new_image_data = image_data.filename[:-4]
    image_data.save(f'uploads/{new_image_data}.png')
    filename = f'{new_image_data}.png'
    
    return redirect(url_for('analyze', filename=filename))


# 사진 분석 로직
@app.route('/analyze', methods = ['GET'])           # 분석 로직 다 적어야 합니다.
def analyze():
    filename = request.args.get('filename')
    print(filename)
    img = Image.open(f'uploads/{filename}')
    # 사진 분석
    send_pic(filename)
    
    return redirect(url_for('send_class', filename=filename))


# 사진 분석하는 로직
def send_pic(filename):
    # 로직 변경의 상황이 있을 수 있습니다.
    # --weights => best.pt의 위치를 넣어주시고 (고정된 값으로)
    # detect.py => 사진이 저장될 위치와 txt가 저장된 위치를 변경하면 됩니다 ㅎㅎ^^;; (save_dir 및 save_txt부분)
    # --source 부분을 filename으로 지정하면 될듯? 일단 사진 저장되는 위치를 파악하고 그 부분을 넣어야합니다.
    image_path = r'C:/Users/smhrd/Yolov7_Detect/uploads'
    # command = f'!python yolov7/detect.py --weights C:\\Users\\smhrd\\Yolov7_Detect\\yolov7\\runs\\train\\try3_now_best\\weights/best.pt --img-size 640 --conf 0.25 --source {image_path}/{filename} --save-txt'
    command = [
        'python', 'yolov7/detect.py',
        '--weights', 'C:/Users/smhrd/Yolov7_Detect/yolov7/runs/train/try3_now_best/weights/best.pt',
        '--img-size', '640',
        '--conf', '0.25',
        '--source', f'{image_path}/{filename}',
        '--save-txt',
        '--save-conf'
    ]
    # 시간이 오래 걸리니까 비동기적으로 명령을 실행하게 해주는
    subprocess.run(command)

@app.route('/send_class/<filename>', methods=['GET'])
def send_class(filename):
    txt_path = r'C:/Users/smhrd/Yolov7_Detect/img_results/labels'
    file_name = filename[:-4]
    with open(f'{txt_path}/{file_name}.txt', 'r') as f:
        data = f.read()
        
    processed_data = process_data(data)
    
    return redirect(url_for('send_image', filename=filename, cnames=','.join(processed_data)))



def process_data(data):
    # 데이터 처리 로직
    
    # txt_string 문자열을 \n 기준으로 분리하여 리스트로 저장
    data = data.split('\n')

    # 리스트를 데이터프레임으로 변환
    df = pd.DataFrame(data)

    # 인덱스 열 추가
    df.index = df.index + 1
    
    # 인덱스 열에 따른 변수 삽입
    lst =[]
    firstNum=df.iloc[0,0][:1]
    secondNum=df.iloc[1,0][:1]
    thirdNum=df.iloc[2,0][:1]
    lst = [firstNum,secondNum,thirdNum]
    
    # 변수의 class number (0~7)에 따라 class명 한국어로 변경
    # 많으면 3개 적으면 그릇 및 음식 명 2개임.
    cName_lst=[]
    for Num in lst:
        if Num == '0':
            Num = '콩밥'
        elif Num == '1':
            Num = '그릇'
        elif Num =='2':
            Num = '닭갈비'
        elif Num =='3':
            Num = '낙지볶음'
        elif Num =='4':
            Num = '돼지고기볶음'
        elif Num =='5':
            Num = '배추김치'
        elif Num =='6':
            Num = '00000000'
        elif Num =='7':
            Num = 'Unknown'
        else:
            Num = ''
        cName_lst.append(Num)
        
    # cName_lst에 ''인 컬럼이 있을 경우 삭제하는 로직
    while '' in cName_lst:
        cName_lst.remove('')
        
    return cName_lst


# 분석된 이미지를 화면에 보여주는 로직 => 성공
@app.route('/send_image', methods=['GET'])
# 여기서 매개변수에 filename을 넣어도 괜찮습니다.
def send_image():
    # 이미지 분석 파일 위치임다.
    filename = request.args.get('filename')
    cnames = request.args.get('cnames')
    if cnames:
        cnames = cnames.split(',')
        print(cnames)
    image_path = r'C:/Users/smhrd/Yolov7_Detect/img_results/'
    origin_image_path = r'C:/Users/smhrd/Yolov7_Detect/uploads'
    # 이건 항상 변할 수 있습니다. png를 넣으면 pgn가 나옴
    img = f'{filename}'
    image = os.path.join(image_path,img)
    
    # DB에 넣기 위해 mem_id를 불러온다면?
    mem_id = f'{mem}'
    analyze_path = image
    origin_path = os.path.join(origin_image_path,img)
    
    # 여기서 path1, path2, mem_id, 식별할 수 있는 친구1 같이 넣기
    
    # 이미지를 로드하고 base64로 인코딩합니다.
    def load_and_encode_image(path):
        with open(path, "rb") as image_file:
            encoded_image = base64.b64encode(image_file.read()).decode('utf-8')
        return encoded_image
    encoded_analyze_image = load_and_encode_image(analyze_path)
    encoded_origin_image = load_and_encode_image(origin_path)
    
    
    # 어짜피 플라스크에 연결된 웹은 1개니까 (내가 연결한거)
    # 결과를 웹으로 보내주는 로직
    return jsonify({
        'analyze_image':encoded_analyze_image,
        'origin_image':encoded_origin_image,
        'cNames':cnames
    })


# 데이터베이스 연결
def conn_database():
    conn = cx_Oracle.connect(
        user='foodhero',
        password='12345',
        dsn=cx_Oracle.makedsn('project-db-stu.ddns.net', 1524, service_name='xe')
    )
    return conn
           
# 데이터베이스 연결 테스트
def test_connection():
    try:
        conn = conn_database()
        print("OracleDB 연결 성공")
    except cx_Oracle.DatabaseError as e:
        print("OracleDB 연결 실패:", e)
    finally:
        if 'conn' in locals():
            conn.close()
test_connection()

# 분석된 사진 및 이전 사진 등 사용자 정보를 저장하는 로직
def save_db_data(mem_id, food_seq, food_img, analyze_img):
    with conn.cursor() as cursor:
        cursor.execute("""
            INSERT INTO MEMBER_DR (MEM_id, req_date, MLD, Food_seq, Food_img, Food_weight, analyze_img)
            VALUES (:mem_id, NULL, NULL, :food_seq, :food_img, NULL, :analyze_img)
        """, {'mem_id': mem_id,
              'food_seq': food_seq,
              'food_img': food_img,
              'analyze_img': analyze_img})
        conn.commit()

# 플라스크 open.
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

OracleDB 연결 성공
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.30.1.97:5000
Press CTRL+C to quit
172.30.1.97 - - [02/May/2023 17:20:03] "POST /upload-image HTTP/1.1" 302 -


0010040000012.png


172.30.1.97 - - [02/May/2023 17:20:14] "GET /analyze?filename=0010040000012.png HTTP/1.1" 302 -
172.30.1.97 - - [02/May/2023 17:20:14] "GET /send_class/0010040000012.png HTTP/1.1" 302 -
172.30.1.97 - - [02/May/2023 17:20:14] "GET /send_image?filename=0010040000012.png&cnames=그릇,배추김치 HTTP/1.1" 200 -


['그릇', '배추김치']


[2023-05-02 17:20:21,313] ERROR in app: Exception on /send_image [GET]
Traceback (most recent call last):
  File "C:\Users\smhrd\anaconda3\lib\site-packages\flask\app.py", line 2190, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\smhrd\anaconda3\lib\site-packages\flask\app.py", line 1486, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\smhrd\anaconda3\lib\site-packages\flask_cors\extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "C:\Users\smhrd\anaconda3\lib\site-packages\flask\app.py", line 1484, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\smhrd\anaconda3\lib\site-packages\flask\app.py", line 1469, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "C:\Users\smhrd\AppData\Local\Temp\ipykernel_11880\1802433576.py", line 166, in send_image
    return send_file(image, mimetype=